[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Hawksight-AI/semantica/blob/main/cookbook/introduction/19_Context_Module.ipynb)

# Context Engineering Module

## Overview

This notebook provides a comprehensive guide to Semantica's **Context Engineering Module** - a powerful system for building context graphs, managing agent memory, retrieving context, and linking entities. You'll learn how to use all classes, functions, and submodules to build intelligent agents with persistent memory and context awareness.

**Documentation**: [API Reference](https://semantica.readthedocs.io/reference/context/)

### Learning Objectives

- **Context Graph Construction**: Build dynamic graphs from entities, relationships, and conversations
- **Agent Memory Management**: Store and retrieve memories with RAG integration
- **Context Retrieval**: Hybrid retrieval combining vector, graph, and memory search
- **Entity Linking**: Link entities across documents with URI assignment
- **Methods Submodule**: Use convenient functions for all context operations
- **Registry System**: Register and use custom context methods
- **Configuration**: Manage module settings and method configurations

### Key Features

- **Context Graphs**: Build graphs from multiple sources (entities, relationships, conversations)
- **Agent Memory**: Persistent memory with vector store and knowledge graph integration
- **Hybrid Retrieval**: Combine vector, graph, and keyword search for optimal context
- **Entity Linking**: Link entities across documents with similarity and graph-based matching
- **Method Registry**: Extensible system for custom context methods
- **Configuration**: Flexible configuration via environment variables and config files

---

## Installation

Install Semantica from PyPI:

```bash
pip install semantica
# Or with all optional dependencies:
pip install semantica[all]
```

---

## Module Structure

The context module is organized into:

- **Main Classes**: `ContextGraphBuilder`, `AgentMemory`, `EntityLinker`, `ContextRetriever`, `AgentContext`
- **Methods Submodule**: Convenient functions for all operations (`methods.build_context_graph`, etc.)
- **Registry Submodule**: Method registry for custom methods (`registry.method_registry`)
- **Config Submodule**: Configuration management (`config.context_config`)

Let's explore each component in detail!


## Step 1: Imports and Setup

First, let's import all the necessary components from the context module.


In [ ]:
# Import main classes
from semantica.context import (
    AgentContext,
    ContextGraphBuilder,
    AgentMemory,
    EntityLinker,
    ContextRetriever,
    ContextNode,
    ContextEdge,
    MemoryItem,
    RetrievedContext
)

# Import submodules
from semantica.context import methods, registry, config

# Import methods functions
from semantica.context.methods import (
    build_context_graph,
    store_memory,
    retrieve_context,
    link_entities,
    get_context_method,
    list_available_methods
)

print("All context module components imported successfully!")
print(f"\nAvailable methods: {list_available_methods()}")


## Step 2: Quick Start with AgentContext

The `AgentContext` class provides a high-level, simplified interface for common use cases. It uses generic method names (`store`, `retrieve`, `forget`, `conversation`) that auto-detect content types and retrieval strategies.


### 2.1 Simple RAG with AgentContext

The simplest way to use the context module for RAG.


In [ ]:
# Initialize AgentContext with vector store
context = AgentContext(vector_store=vs)

# Store a memory
memory_id = context.store(
    "User likes Python programming and machine learning",
    conversation_id="conv1",
    user_id="user123"
)

print(f"Stored memory with ID: {memory_id}")

# Retrieve context
results = context.retrieve(
    "Python programming",
    max_results=5,
    use_graph=False  # Boolean flag: force vector-only retrieval
)

print(f"\nRetrieved {len(results)} results")
for i, result in enumerate(results, 1):
    print(f"\n{i}. Content: {result['content'][:60]}...")
    print(f"   Score: {result['score']:.2f}")
    print(f"   Source: {result['source']}")


### 2.2 GraphRAG with AgentContext

Store documents and retrieve with graph context.


In [ ]:
# Initialize with vector store and knowledge graph
context = AgentContext(
    vector_store=vs,
    knowledge_graph=kg,
    use_graph_expansion=True,  # Boolean flag
    max_expansion_hops=2
)

# Store documents (auto-builds graph)
documents = [
    "Python is a programming language used for machine learning and data science",
    "TensorFlow and PyTorch are popular machine learning frameworks",
    "Machine learning involves training models on data to make predictions"
]

stats = context.store(
    documents,
    extract_entities=True,      # Boolean flag: extract entities
    extract_relationships=True,  # Boolean flag: extract relationships
    link_entities=True          # Boolean flag: link entities across documents
)

print(f"Stored {stats['stored_count']} documents")
print(f"Built graph with {stats['graph_nodes']} nodes and {stats['graph_edges']} edges")

# Retrieve with graph context (auto-detects GraphRAG)
results = context.retrieve(
    "Python machine learning",
    max_results=5,
    use_graph=None,              # Auto-detect (uses graph since knowledge_graph available)
    include_entities=True,       # Boolean flag: include related entities
    include_relationships=False, # Boolean flag: don't include relationships
    expand_graph=True            # Boolean flag: use graph expansion
)

print(f"\nRetrieved {len(results)} results with graph context")
for i, result in enumerate(results, 1):
    print(f"\n{i}. Content: {result['content'][:60]}...")
    print(f"   Score: {result['score']:.2f}")
    print(f"   Source: {result['source']}")
    if result.get('related_entities'):
        print(f"   Related entities: {len(result['related_entities'])}")


### 2.3 Conversation Management with AgentContext

Manage conversation history easily.


In [ ]:
context = AgentContext(vector_store=vs, retention_days=30)

# Store multiple memories in a conversation
context.store("Hello, I'm interested in Python", conversation_id="conv1", user_id="user123")
context.store("What can you tell me about machine learning?", conversation_id="conv1", user_id="user123")
context.store("I prefer TensorFlow over PyTorch", conversation_id="conv1", user_id="user123")

# Get conversation history
history = context.conversation(
    "conv1",
    reverse=True,           # Boolean flag: most recent first
    include_metadata=True   # Boolean flag: include full metadata
)

print(f"Retrieved {len(history)} items from conversation")
for item in history:
    print(f"   - {item.get('timestamp', 'N/A')}: {item['content']}")

# Retrieve context from conversation
results = context.retrieve(
    "Python",
    conversation_id="conv1",  # Filter by conversation
    max_results=3
)

print(f"\nRetrieved {len(results)} results from conversation")


### 2.4 Additional Memory Management Methods

AgentContext provides many additional methods for memory management, search, and analytics.


In [ ]:
# Memory Management Methods
print("=== Memory Management ===")

# Check if memory exists
exists = context.exists(memory_id) if memory_id else False
print(f"Memory exists: {exists}")

# Get memory count
total = context.count()
conv_count = context.count(conversation_id="conv1")
print(f"Total memories: {total}, Conversation memories: {conv_count}")

# Get memory by ID
if memory_id:
    memory = context.get(memory_id)
    if memory:
        print(f"Retrieved memory: {memory['content'][:50]}...")

# Update memory
# context.update(memory_id, content="Updated content", metadata={"key": "value"})

# List memories with pagination
memories = context.list(conversation_id="conv1", limit=10, offset=0)
print(f"Listed {len(memories)} memories")

# Batch operations
# ids = context.batch_store(["Item 1", "Item 2", "Item 3"])
# deleted = context.batch_delete(["mem1", "mem2"])

# Search Methods
print("\n=== Search Methods ===")
results = context.search("Python", max_results=5)
similar = context.find_similar("Python programming", limit=5)
context_data = context.get_context("Python", max_results=5)
print(f"Search results: {len(results)}, Similar: {len(similar)}, Context: {len(context_data)}")

# Conversation Methods
print("\n=== Conversation Methods ===")
conversations = context.list_conversations(user_id="user123", limit=50)
summary = context.conversation_summary("conv1")
print(f"Conversations: {len(conversations)}, Summary: {summary.get('message_count', 0)} messages")

# Export/Import
print("\n=== Export/Import ===")
# backup_data = context.backup()
# restored = context.restore(backup_data)

# Statistics
print("\n=== Statistics ===")
stats = context.stats()
health = context.health()
usage = context.usage_stats(period='day')
print(f"Stats: {stats.get('total_items', 0)} items")
print(f"Health: {health['status']}")
print(f"Usage: {usage.get('recent_memories', 0)} recent memories")


---

## Step 3: Context Graph Construction

The `ContextGraphBuilder` class allows you to build context graphs from various sources. Let's explore different ways to construct graphs.

### 3.1 Building from Entities and Relationships

The most straightforward way to build a context graph is from entities and relationships.


In [ ]:
# Initialize the builder
builder = ContextGraphBuilder()

# Define entities
entities = [
    {"id": "e1", "text": "Python", "type": "PROGRAMMING_LANGUAGE", "confidence": 0.95},
    {"id": "e2", "text": "Machine Learning", "type": "CONCEPT", "confidence": 0.9},
    {"id": "e3", "text": "TensorFlow", "type": "FRAMEWORK", "confidence": 0.92},
    {"id": "e4", "text": "PyTorch", "type": "FRAMEWORK", "confidence": 0.91},
]

# Define relationships
relationships = [
    {"source_id": "e1", "target_id": "e2", "type": "used_for", "confidence": 0.9},
    {"source_id": "e3", "target_id": "e2", "type": "implements", "confidence": 0.95},
    {"source_id": "e4", "target_id": "e2", "type": "implements", "confidence": 0.94},
    {"source_id": "e3", "target_id": "e1", "type": "built_with", "confidence": 0.88},
    {"source_id": "e4", "target_id": "e1", "type": "built_with", "confidence": 0.87},
]

# Build the graph
graph = builder.build_from_entities_and_relationships(entities, relationships)

print(f"Graph built successfully!")
print(f"Statistics:")
print(f"   - Nodes: {graph['statistics']['node_count']}")
print(f"   - Edges: {graph['statistics']['edge_count']}")
print(f"   - Node types: {list(graph['statistics']['node_types'].keys())}")
print(f"   - Edge types: {list(graph['statistics']['edge_types'].keys())}")

# Display some nodes
print(f"\nSample Nodes:")
for node_id, node_data in list(graph['nodes'].items())[:3]:
    print(f"   - {node_id}: {node_data['text']} ({node_data['type']})")

# Display some edges
print(f"\nSample Edges:")
for edge in list(graph['edges'])[:3]:
    print(f"   - {edge['source_id']} --[{edge['type']}]--> {edge['target_id']}")


### 3.2 Building from Conversations

You can also build graphs from conversation data, which automatically extracts entities, relationships, intents, and sentiments.


In [ ]:
# Define conversations
conversations = [
    {
        "id": "conv1",
        "content": "User asked about Python programming and machine learning",
        "timestamp": "2024-01-01T10:00:00",
        "entities": [
            {"id": "e1", "text": "Python", "type": "PROGRAMMING_LANGUAGE"},
            {"id": "e2", "text": "Machine Learning", "type": "CONCEPT"}
        ],
        "relationships": [
            {"source_id": "e1", "target_id": "e2", "type": "used_for"}
        ]
    },
    {
        "id": "conv2",
        "content": "User asked about TensorFlow and PyTorch frameworks",
        "timestamp": "2024-01-01T11:00:00",
        "entities": [
            {"id": "e3", "text": "TensorFlow", "type": "FRAMEWORK"},
            {"id": "e4", "text": "PyTorch", "type": "FRAMEWORK"}
        ],
        "relationships": [
            {"source_id": "e3", "target_id": "e4", "type": "related_to"}
        ]
    }
]

# Build graph from conversations with additional features
graph = builder.build_from_conversations(
    conversations,
    link_entities=True,      # Link similar entities across conversations
    extract_intents=True,     # Extract conversation intents
    extract_sentiments=True   # Extract sentiment information
)

print(f"Graph built from {len(conversations)} conversations!")
print(f"Statistics:")
print(f"   - Nodes: {graph['statistics']['node_count']}")
print(f"   - Edges: {graph['statistics']['edge_count']}")
print(f"   - Conversations processed: {len(conversations)}")


### 3.3 Additional Graph Construction Methods

ContextGraphBuilder provides high-level methods for easy graph construction.


In [ ]:
# High-level construction methods
builder = ContextGraphBuilder()

# Build from text
graph = builder.from_text("Python is used for machine learning")
print(f"From text: {graph.get('statistics', {}).get('node_count', 0)} nodes")

# Build from documents
docs = ["Doc 1 about Python", "Doc 2 about ML"]
graph = builder.from_documents(docs)
print(f"From documents: {graph.get('statistics', {}).get('node_count', 0)} nodes")

# Simple add method
stats = builder.add(entities=entities, relationships=relationships)
print(f"Add method: {stats.get('node_count', 0)} nodes, {stats.get('edge_count', 0)} edges")

# Query methods
results = builder.find("Python")
node = builder.find_node("e1")
nodes = builder.find_nodes(node_type="PROGRAMMING_LANGUAGE")
edges = builder.find_edges(edge_type="used_for")
path = builder.find_path("e1", "e2", max_hops=5)

print(f"\nQuery results:")
print(f"   Find: {len(results)} results")
print(f"   Find node: {node is not None}")
print(f"   Find nodes: {len(nodes)} nodes")
print(f"   Find edges: {len(edges)} edges")
print(f"   Find path: {len(path)} nodes in path")

# Graph operations
subgraph = builder.get_subgraph(["e1", "e2"])
cloned = builder.clone()
stats = builder.stats()
node_count = builder.node_count(node_type="PROGRAMMING_LANGUAGE")
edge_count = builder.edge_count(edge_type="used_for")
density = builder.density()

print(f"\nGraph operations:")
print(f"   Subgraph: {subgraph.get('statistics', {}).get('node_count', 0)} nodes")
print(f"   Cloned: {cloned.stats().get('node_count', 0)} nodes")
print(f"   Stats: {stats.get('node_count', 0)} nodes, {stats.get('edge_count', 0)} edges")
print(f"   Node count: {node_count}")
print(f"   Edge count: {edge_count}")
print(f"   Density: {density:.3f}")


## Step 4: Agent Memory Management

The `AgentMemory` class provides persistent memory for agents with RAG integration. It stores memories with vector embeddings and integrates with knowledge graphs.


### 4.1 Initializing Agent Memory

First, we need to set up a vector store and knowledge graph. For demonstration, we'll use mock objects.


In [ ]:
# Note: In a real scenario, you would use actual vector store and knowledge graph instances
# For demonstration, we'll create mock objects

class MockVectorStore:
    """Mock vector store for demonstration."""
    def __init__(self):
        self.vectors = {}
        self.metadata = {}
    
    def add(self, ids, embeddings, metadatas=None):
        for i, id_val in enumerate(ids):
            self.vectors[id_val] = embeddings[i]
            if metadatas:
                self.metadata[id_val] = metadatas[i]
    
    def query(self, query_embeddings, n_results=5, where=None):
        # Simplified mock query
        results = []
        for i, (id_val, vector) in enumerate(list(self.vectors.items())[:n_results]):
            results.append({
                'id': id_val,
                'score': 0.9 - i * 0.1,
                'metadata': self.metadata.get(id_val, {})
            })
        return {'ids': [[r['id'] for r in results]], 'metadatas': [[r['metadata'] for r in results]]}

class MockKnowledgeGraph:
    """Mock knowledge graph for demonstration."""
    def __init__(self):
        self.entities = {}
        self.relationships = []
        self.nodes = []
        self.edges = []
    
    def add_entity(self, entity_id, entity_data):
        self.entities[entity_id] = entity_data
    
    def add_relationship(self, source, target, rel_type):
        self.relationships.append({
            'source_id': source,
            'target_id': target,
            'type': rel_type
        })

# Initialize mock stores
vs = MockVectorStore()
kg = MockKnowledgeGraph()

# Initialize AgentMemory
memory = AgentMemory(
    vector_store=vs,
    knowledge_graph=kg,
    retention_policy="30_days",  # Keep memories for 30 days
    max_memory_size=10000         # Maximum number of memories
)

print("AgentMemory initialized successfully!")
print(f"Configuration:")
print(f"   - Retention policy: 30_days")
print(f"   - Max memory size: 10000")


### 4.2 Storing Memories

Store memories with metadata and associated entities.


In [ ]:
# Store a memory with metadata
memory_id = memory.store(
    "User asked about Python programming and machine learning",
    metadata={
        "type": "conversation",
        "conversation_id": "conv_123",
        "user_id": "user_456",
        "timestamp": "2024-01-01T10:00:00"
    },
    entities=[
        {"id": "e1", "text": "Python", "type": "PROGRAMMING_LANGUAGE"},
        {"id": "e2", "text": "Machine Learning", "type": "CONCEPT"}
    ]
)

print(f"Memory stored with ID: {memory_id}")

# Store another memory
memory_id_2 = memory.store(
    "User asked about TensorFlow framework for deep learning",
    metadata={
        "type": "conversation",
        "conversation_id": "conv_124",
        "user_id": "user_456"
    },
    entities=[
        {"id": "e3", "text": "TensorFlow", "type": "FRAMEWORK"},
        {"id": "e2", "text": "Deep Learning", "type": "CONCEPT"}
    ]
)

print(f"Second memory stored with ID: {memory_id_2}")

# Get statistics
stats = memory.get_statistics()
print(f"\nMemory Statistics:")
print(f"   - Total items: {stats['total_items']}")
print(f"   - Items by type: {stats.get('items_by_type', {})}")


### 4.3 Additional Memory Management Methods

AgentMemory provides many additional methods for memory operations.


In [ ]:
# Basic operations
print("=== Basic Operations ===")
exists = memory.exists(memory_id) if memory_id else False
count = memory.count()
mem = memory.get(memory_id) if memory_id else None
print(f"Exists: {exists}, Count: {count}, Retrieved: {mem is not None}")

# Search methods
print("\n=== Search Methods ===")
results = memory.search("Python", max_results=5)
similar = memory.find_similar("Python programming", limit=5)
by_entity = memory.find_by_entity("e1", limit=10)
by_relationship = memory.find_by_relationship("used_for", limit=10)
print(f"Search: {len(results)}, Similar: {len(similar)}, By entity: {len(by_entity)}, By relationship: {len(by_relationship)}")

# List and filter methods
print("\n=== List and Filter Methods ===")
memories = memory.list(conversation_id="conv_123", limit=10)
conv_memories = memory.get_by_conversation("conv_123", limit=100)
user_memories = memory.get_by_user("user_456", limit=100)
recent = memory.get_recent(limit=10)
print(f"List: {len(memories)}, Conversation: {len(conv_memories)}, User: {len(user_memories)}, Recent: {len(recent)}")

# Batch operations
print("\n=== Batch Operations ===")
# ids = memory.batch_store(["Item 1", "Item 2"])
# deleted = memory.batch_delete(["mem1", "mem2"])
# updated = memory.batch_update([{"memory_id": "mem1", "content": "New"}])

# Statistics
print("\n=== Statistics ===")
stats = memory.stats()
type_counts = memory.count_by_type()
user_counts = memory.count_by_user()
conv_counts = memory.count_by_conversation()
print(f"Stats: {stats.get('total_items', 0)} items")
print(f"By type: {type_counts}")
print(f"By user: {user_counts}")
print(f"By conversation: {conv_counts}")


### 4.4 Conversation History

Retrieve conversation history for a specific conversation.


In [ ]:
# Get conversation history
history = memory.get_conversation_history(
    conversation_id="conv_123",
    max_items=100
)

print(f"Retrieved {len(history)} items from conversation history")
for item in history:
    print(f"   - {item.get('timestamp', 'N/A')}: {item.get('content', '')[:50]}...")


### 4.5 Memory Management

Delete and manage memories with various filters.


In [ ]:
# Delete a specific memory
if memory_id_2:
    memory.delete(memory_id_2)
    print(f"Deleted memory: {memory_id_2}")

# Clear memories by filters
deleted_count = memory.clear(
    type="conversation",
    start_date="2024-01-01",
    end_date="2024-12-31"
)

print(f"Cleared {deleted_count} memories matching filters")

# Get updated statistics
stats = memory.get_statistics()
print(f"\nUpdated Statistics:")
print(f"   - Total items: {stats['total_items']}")


## Step 5: Context Retrieval

The `ContextRetriever` class provides hybrid context retrieval combining vector search, graph traversal, and memory search.


### 5.1 Basic Context Retrieval

Initialize and use the context retriever for hybrid retrieval.


In [ ]:
# Initialize ContextRetriever
retriever = ContextRetriever(
    memory_store=memory,
    knowledge_graph=kg,
    vector_store=vs,
    use_graph_expansion=True,  # Enable graph expansion for related entities
    max_expansion_hops=2        # Maximum hops for graph expansion
)

# Retrieve context
results = retriever.retrieve(
    "Python programming",
    max_results=5,
    min_relevance_score=0.5
)

print(f"Retrieved {len(results)} context items")
print(f"\nResults:")
for i, result in enumerate(results, 1):
    print(f"\n{i}. Content: {result.content[:60]}...")
    print(f"   Score: {result.score:.2f}")
    print(f"   Source: {result.source}")
    print(f"   Related entities: {len(result.related_entities)}")
    if result.related_entities:
        print(f"   - {', '.join([e.get('content', '') for e in result.related_entities[:3]])}")


### 5.2 Additional Retrieval Methods

ContextRetriever provides many additional methods for different retrieval strategies.


In [ ]:
# Search methods
print("=== Search Methods ===")
results = retriever.search("Python", max_results=5)
vector_results = retriever.vector_search("Python")
graph_results = retriever.graph_search("Python")
memory_results = retriever.memory_search("Python")
hybrid_results = retriever.hybrid_search("Python")
print(f"Search: {len(results)}, Vector: {len(vector_results)}, Graph: {len(graph_results)}, Memory: {len(memory_results)}, Hybrid: {len(hybrid_results)}")

# Advanced retrieval
print("\n=== Advanced Retrieval ===")
similar = retriever.find_similar("Python programming", limit=5)
context = retriever.get_context("Python", max_results=5)
expanded = retriever.expand_query("Python", max_hops=2)
related = retriever.get_related("e1", max_hops=2) if hasattr(kg, 'nodes') else []
path = retriever.get_path("e1", "e2", max_hops=5) if hasattr(kg, 'nodes') else []
print(f"Similar: {len(similar)}, Context: {len(context)}, Expanded: {len(expanded)}, Related: {len(related)}, Path: {len(path)}")

# Filter methods
print("\n=== Filter Methods ===")
# by_entity = retriever.filter_by_entity("e1", "Python")
# by_type = retriever.filter_by_type("PROGRAMMING_LANGUAGE", "Python")
# by_date = retriever.filter_by_date("2024-01-01", "2024-12-31", "Python")
# by_score = retriever.filter_by_score(0.7, "Python")

# Batch operations
print("\n=== Batch Operations ===")
# batch_results = retriever.batch_search(["Python", "Java", "C++"])
# batch_contexts = retriever.batch_get_context(["Python", "Java"], max_results=5)


## Step 6: Entity Linking

The `EntityLinker` class links entities across documents with URI assignment and similarity matching.


### 6.1 Basic Entity Linking

Link entities in text and assign URIs.


In [ ]:
# Initialize EntityLinker
linker = EntityLinker(
    knowledge_graph=kg,
    similarity_threshold=0.8,
    base_uri="https://semantica.dev/entity/"
)

# Assign URI to an entity
uri = linker.assign_uri(
    "entity_1",
    "Python",
    "PROGRAMMING_LANGUAGE"
)
print(f"Assigned URI: {uri}")

# Link entities in text
entities = [
    {"id": "e1", "text": "Python", "type": "PROGRAMMING_LANGUAGE"},
    {"id": "e2", "text": "Machine Learning", "type": "CONCEPT"},
]

linked_entities = linker.link(
    "Python is used for machine learning",
    entities=entities
)

print(f"\nLinked {len(linked_entities)} entities")
for entity in linked_entities:
    print(f"   - {entity.text}: {entity.uri}")
    print(f"     Linked to {len(entity.linked_entities)} entities")


### 6.2 Additional Entity Linking Methods

EntityLinker provides many additional methods for linking and searching entities.


In [ ]:
# Create explicit link between entities
linker.link_entities(
    "entity_1",
    "entity_2",
    link_type="related_to",
    confidence=0.9,
    source="manual"
)

# Get entity links
links = linker.get_entity_links("entity_1")
print(f"✅ Found {len(links)} links for entity_1")
for link in links:
    print(f"   - {link.source_entity_id} --[{link.link_type}]--> {link.target_entity_id}")

# Get entity URI
uri = linker.get_entity_uri("entity_1")
if uri:
    print(f"\n✅ Entity URI: {uri}")


### 5.3 Finding Similar Entities

Find similar entities using similarity matching.


In [ ]:
# Find similar entities
similar = linker.find_similar_entities(
    "Python",
    entity_type="PROGRAMMING_LANGUAGE",
    threshold=0.8
)

print(f"✅ Found {len(similar)} similar entities")
for entity_id, similarity in similar:
    print(f"   - {entity_id}: {similarity:.2f} similarity")


### 5.4 Building Entity Web

Build a complete entity web showing all connections.


In [ ]:
# Link multiple entities
linker.link_entities("e1", "e2", "related_to", confidence=0.9)
linker.link_entities("e2", "e3", "related_to", confidence=0.85)

# Build entity web
web = linker.build_entity_web()

print(f"✅ Entity Web Statistics:")
print(f"   - Total entities: {web['statistics']['total_entities']}")
print(f"   - Total links: {web['statistics']['total_links']}")

for entity_id, info in list(web['entities'].items())[:5]:
    print(f"\n   Entity {entity_id}:")
    print(f"     - URI: {info.get('uri', 'N/A')}")
    print(f"     - Links: {info.get('links', 0)}")


## 🎯 Step 6: Methods Submodule

The `methods` submodule provides convenient functions for all context operations. These functions use the method registry system and support multiple algorithms.


### 6.1 Building Context Graphs with Methods

Use the `build_context_graph` function with different methods.


In [ ]:
# Build from entities and relationships
graph = methods.build_context_graph(
    entities=entities,
    relationships=relationships,
    method="entities_relationships"
)

print(f"✅ Built graph using 'entities_relationships' method")
print(f"   - Nodes: {graph['statistics']['node_count']}")
print(f"   - Edges: {graph['statistics']['edge_count']}")

# Build from conversations
graph = methods.build_context_graph(
    conversations=conversations,
    method="conversations"
)

print(f"\n✅ Built graph using 'conversations' method")
print(f"   - Nodes: {graph['statistics']['node_count']}")

# Hybrid construction
graph = methods.build_context_graph(
    entities=entities,
    relationships=relationships,
    conversations=conversations,
    method="hybrid"
)

print(f"\n✅ Built graph using 'hybrid' method")
print(f"   - Nodes: {graph['statistics']['node_count']}")


### 6.2 Storing Memories with Methods

Use the `store_memory` function with different storage methods.


In [ ]:
# Store memory using method
memory_id = methods.store_memory(
    "User asked about Python programming",
    vector_store=vs,
    knowledge_graph=kg,
    method="store",
    metadata={"type": "conversation"}
)

print(f"✅ Stored memory with ID: {memory_id}")

# Store conversation memory
memory_id = methods.store_memory(
    "User asked about machine learning frameworks",
    vector_store=vs,
    knowledge_graph=kg,
    method="conversation",
    metadata={"conversation_id": "conv_125"}
)

print(f"✅ Stored conversation memory with ID: {memory_id}")


### 6.3 Retrieving Context with Methods

Use the `retrieve_context` function with different retrieval methods.


In [ ]:
# Vector-based retrieval only
results = methods.retrieve_context(
    "Python programming",
    vector_store=vs,
    method="vector",
    max_results=5
)
print(f"✅ Vector retrieval: {len(results)} results")

# Graph-based retrieval only
results = methods.retrieve_context(
    "Python programming",
    knowledge_graph=kg,
    method="graph",
    max_results=5
)
print(f"✅ Graph retrieval: {len(results)} results")

# Memory-based retrieval only
results = methods.retrieve_context(
    "Python programming",
    memory_store=memory,
    method="memory",
    max_results=5
)
print(f"✅ Memory retrieval: {len(results)} results")

# Hybrid retrieval (all sources)
results = methods.retrieve_context(
    "Python programming",
    memory_store=memory,
    knowledge_graph=kg,
    vector_store=vs,
    method="hybrid",
    max_results=5
)
print(f"✅ Hybrid retrieval: {len(results)} results")


### 6.4 Linking Entities with Methods

Use the `link_entities` function with different linking methods.


In [ ]:
# URI assignment only
linked = methods.link_entities(
    entities,
    method="uri"
)
print(f"✅ URI linking: {len(linked)} entities linked")

# Similarity-based linking
linked = methods.link_entities(
    entities,
    knowledge_graph=kg,
    method="similarity"
)
print(f"✅ Similarity linking: {len(linked)} entities linked")

# Knowledge graph-based linking
linked = methods.link_entities(
    entities,
    knowledge_graph=kg,
    method="knowledge_graph"
)
print(f"✅ Knowledge graph linking: {len(linked)} entities linked")

# Cross-document linking
linked = methods.link_entities(
    entities,
    knowledge_graph=kg,
    method="cross_document",
    context=[{"source": "doc1"}, {"source": "doc2"}]
)
print(f"✅ Cross-document linking: {len(linked)} entities linked")


### 6.5 Listing Available Methods

Discover all available methods for different tasks.


In [ ]:
# List all available methods
all_methods = methods.list_available_methods()
print("📋 All Available Methods:")
for task, method_list in all_methods.items():
    print(f"   {task}: {method_list}")

# List methods for specific task
graph_methods = methods.list_available_methods("graph")
print(f"\n📊 Graph Methods: {graph_methods}")

# Get a specific method
custom_method = methods.get_context_method("graph", "entities_relationships")
if custom_method:
    print(f"\n✅ Retrieved method: {custom_method.__name__}")


## 🎛️ Step 7: Registry Submodule

The `registry` submodule allows you to register custom context methods for extensibility.


### 7.1 Registering Custom Methods

Register your own custom context methods.


In [ ]:
# Define a custom graph builder function
def custom_graph_builder(entities, relationships, **kwargs):
    """Custom graph building method with additional processing."""
    # Custom implementation
    nodes = {}
    edges = []
    
    # Process entities
    for entity in entities:
        nodes[entity['id']] = {
            'text': entity.get('text', ''),
            'type': entity.get('type', ''),
            'confidence': entity.get('confidence', 1.0)
        }
    
    # Process relationships
    for rel in relationships:
        edges.append({
            'source_id': rel['source_id'],
            'target_id': rel['target_id'],
            'type': rel['type'],
            'weight': rel.get('confidence', 1.0)
        })
    
    return {
        'nodes': nodes,
        'edges': edges,
        'statistics': {
            'node_count': len(nodes),
            'edge_count': len(edges)
        }
    }

# Register custom method
registry.method_registry.register("graph", "custom_builder", custom_graph_builder)

print("✅ Registered custom method: 'custom_builder'")

# Use custom method
graph = methods.build_context_graph(
    entities=entities,
    relationships=relationships,
    method="custom_builder"
)

print(f"✅ Built graph using custom method")
print(f"   - Nodes: {graph['statistics']['node_count']}")
print(f"   - Edges: {graph['statistics']['edge_count']}")


### 7.2 Managing Registered Methods

List and manage registered methods.


In [ ]:
# List all registered methods for a task
methods_list = registry.method_registry.list_all("graph")
print(f"📋 Registered graph methods: {methods_list}")

# Get a registered method
retrieved_method = registry.method_registry.get("graph", "custom_builder")
if retrieved_method:
    print(f"✅ Retrieved method: {retrieved_method.__name__}")

# Unregister a method
registry.method_registry.unregister("graph", "custom_builder")
print(f"✅ Unregistered method: 'custom_builder'")

# Verify unregistration
methods_list = registry.method_registry.list_all("graph")
print(f"📋 Updated methods list: {methods_list}")


## ⚙️ Step 8: Configuration Submodule

The `config` submodule provides centralized configuration management for the context module.


### 8.1 Basic Configuration

Get and set configuration values.


In [ ]:
# Get configuration values
retention = config.context_config.get("retention_policy", default="unlimited")
max_size = config.context_config.get("max_memory_size", default=10000)

print(f"📋 Current Configuration:")
print(f"   - Retention policy: {retention}")
print(f"   - Max memory size: {max_size}")

# Set configuration values
config.context_config.set("retention_policy", "30_days")
config.context_config.set("max_memory_size", 5000)

print(f"\n✅ Updated Configuration:")
print(f"   - Retention policy: {config.context_config.get('retention_policy')}")
print(f"   - Max memory size: {config.context_config.get('max_memory_size')}")


### 8.2 Method-Specific Configuration

Configure settings for specific methods.


In [ ]:
# Set method-specific configuration
config.context_config.set_method_config("graph", {
    "extract_entities": True,
    "extract_relationships": True,
    "max_depth": 3
})

# Get method-specific configuration
method_config = config.context_config.get_method_config("graph")
print(f"📋 Graph Method Configuration:")
print(f"   {method_config}")

# Set memory method configuration
config.context_config.set_method_config("memory", {
    "retention_policy": "unlimited",
    "embedding_dimensions": 1536
})

memory_config = config.context_config.get_method_config("memory")
print(f"\n📋 Memory Method Configuration:")
print(f"   {memory_config}")


### 8.3 Getting All Configuration

Retrieve all configuration values at once.


In [ ]:
# Get all configurations
all_configs = config.context_config.get_all()

print(f"📋 All Configuration Values:")
for key, value in all_configs.items():
    if isinstance(value, dict):
        print(f"   {key}:")
        for sub_key, sub_value in value.items():
            print(f"      {sub_key}: {sub_value}")
    else:
        print(f"   {key}: {value}")


### 8.4 Environment Variables

Configuration can also be set via environment variables. The module automatically reads:
- `CONTEXT_RETENTION_POLICY`
- `CONTEXT_MAX_MEMORY_SIZE`
- `CONTEXT_SIMILARITY_THRESHOLD`
- And more...

```bash
# Example environment variable setup
export CONTEXT_RETENTION_POLICY=30_days
export CONTEXT_MAX_MEMORY_SIZE=5000
export CONTEXT_SIMILARITY_THRESHOLD=0.8
```


## 🚀 Step 9: Advanced Examples

Let's combine everything we've learned into complete workflows for building intelligent agents with context awareness.


### 9.1 Complete Agent Context Workflow

A complete workflow from building context graphs to retrieving context for agent responses.


In [ ]:
# Step 1: Build context graph from entities and relationships
builder = ContextGraphBuilder()
graph = builder.build_from_entities_and_relationships(entities, relationships)

print(f"✅ Step 1: Built context graph with {graph['statistics']['node_count']} nodes")

# Step 2: Initialize agent memory
memory = AgentMemory(vector_store=vs, knowledge_graph=kg)

# Step 3: Store conversation in memory
conversation_id = "conv_126"
memory_id = memory.store(
    "User asked about Python programming and machine learning frameworks",
    metadata={
        "type": "conversation",
        "conversation_id": conversation_id,
        "user_id": "user_789",
        "timestamp": "2024-01-01T12:00:00"
    },
    entities=[
        {"id": "e1", "text": "Python", "type": "PROGRAMMING_LANGUAGE"},
        {"id": "e2", "text": "Machine Learning", "type": "CONCEPT"}
    ]
)

print(f"✅ Step 2-3: Stored conversation memory with ID: {memory_id}")

# Step 4: Link entities
linker = EntityLinker(knowledge_graph=kg)
linked = linker.link(
    "Python is used for machine learning",
    entities=entities
)

print(f"✅ Step 4: Linked {len(linked)} entities")

# Step 5: Retrieve context for agent response
retriever = ContextRetriever(
    memory_store=memory,
    knowledge_graph=kg,
    vector_store=vs
)

results = retriever.retrieve("Python programming", max_results=5)

print(f"✅ Step 5: Retrieved {len(results)} context items")
print(f"\n📋 Context for Agent Response:")
for i, result in enumerate(results[:3], 1):
    print(f"\n{i}. {result.content[:60]}...")
    print(f"   Score: {result.score:.2f}, Source: {result.source}")


In [ ]:
# Initialize retriever with multiple sources
retriever = ContextRetriever(
    memory_store=memory,
    knowledge_graph=kg,
    vector_store=vs,
    hybrid_alpha=0.5,  # Balance between vector and graph
    use_graph_expansion=True,
    max_expansion_hops=2
)

# Retrieve with hybrid approach
results = retriever.retrieve(
    "Python machine learning frameworks",
    max_results=10,
    use_graph_expansion=True,
    max_hops=2
)

print(f"✅ Retrieved {len(results)} results from multiple sources")
print(f"\n📋 Multi-Source Results:")
for i, result in enumerate(results[:5], 1):
    print(f"\n{i}. Source: {result.source}")
    print(f"   Content: {result.content[:80]}...")
    print(f"   Score: {result.score:.2f}")
    print(f"   Related entities: {len(result.related_entities)}")
    if result.related_entities:
        print(f"   - {', '.join([e.get('content', '') for e in result.related_entities[:3]])}")


### 9.3 Conversation-Based Context Building

Build context graphs from conversation history and maintain conversation context.


In [ ]:
# Process multiple conversations
conversations = [
    {
        "id": "conv1",
        "content": "User asked about Python programming",
        "timestamp": "2024-01-01T10:00:00",
        "entities": [{"id": "e1", "text": "Python", "type": "PROGRAMMING_LANGUAGE"}]
    },
    {
        "id": "conv2",
        "content": "User asked about machine learning",
        "timestamp": "2024-01-01T11:00:00",
        "entities": [{"id": "e2", "text": "Machine Learning", "type": "CONCEPT"}]
    },
    {
        "id": "conv3",
        "content": "User asked about TensorFlow and PyTorch",
        "timestamp": "2024-01-01T12:00:00",
        "entities": [
            {"id": "e3", "text": "TensorFlow", "type": "FRAMEWORK"},
            {"id": "e4", "text": "PyTorch", "type": "FRAMEWORK"}
        ]
    }
]

# Build graph from conversations
builder = ContextGraphBuilder()
graph = builder.build_from_conversations(
    conversations,
    link_entities=True,
    extract_intents=True
)

print(f"✅ Built graph from {len(conversations)} conversations")
print(f"   - Nodes: {graph['statistics']['node_count']}")
print(f"   - Edges: {graph['statistics']['edge_count']}")

# Store conversations in memory
for conv in conversations:
    memory.store(
        conv["content"],
        metadata={
            "type": "conversation",
            "conversation_id": conv["id"],
            "timestamp": conv["timestamp"]
        },
        entities=conv.get("entities", [])
    )

print(f"\n✅ Stored {len(conversations)} conversations in memory")

# Retrieve conversation history
history = memory.get_conversation_history(conversation_id="conv1")
print(f"\n📋 Conversation History for conv1: {len(history)} items")


### 9.4 Entity Web Construction

Build a complete entity web showing all entity connections and relationships.


In [ ]:
# Initialize linker
linker = EntityLinker(
    knowledge_graph=kg,
    similarity_threshold=0.8
)

# Define entities
entities = [
    {"id": "e1", "text": "Python", "type": "PROGRAMMING_LANGUAGE"},
    {"id": "e2", "text": "Machine Learning", "type": "CONCEPT"},
    {"id": "e3", "text": "TensorFlow", "type": "FRAMEWORK"},
    {"id": "e4", "text": "PyTorch", "type": "FRAMEWORK"},
    {"id": "e5", "text": "Deep Learning", "type": "CONCEPT"},
]

# Link entities
linked = linker.link("", entities=entities)

# Create explicit links
linker.link_entities("e1", "e2", "used_for", confidence=0.9)
linker.link_entities("e3", "e2", "implements", confidence=0.95)
linker.link_entities("e4", "e2", "implements", confidence=0.94)
linker.link_entities("e3", "e5", "implements", confidence=0.92)
linker.link_entities("e4", "e5", "implements", confidence=0.91)
linker.link_entities("e3", "e4", "related_to", confidence=0.8)

# Build entity web
web = linker.build_entity_web()

print(f"✅ Entity Web Statistics:")
print(f"   - Total entities: {web['statistics']['total_entities']}")
print(f"   - Total links: {web['statistics']['total_links']}")

print(f"\n📋 Entity Details:")
for entity_id, info in web['entities'].items():
    print(f"\n   Entity {entity_id}:")
    print(f"     - URI: {info.get('uri', 'N/A')}")
    print(f"     - Links: {info.get('links', 0)}")
    print(f"     - Type: {info.get('type', 'N/A')}")


### 9.5 Using Methods for Complete Workflow

Use the methods submodule for a streamlined workflow.


In [ ]:
# Complete workflow using methods submodule

# 1. Build context graph
graph = methods.build_context_graph(
    entities=entities,
    relationships=relationships,
    method="hybrid"
)
print(f"✅ Step 1: Built graph with {graph['statistics']['node_count']} nodes")

# 2. Store memory
memory_id = methods.store_memory(
    "User conversation about Python and machine learning",
    vector_store=vs,
    knowledge_graph=kg,
    method="store",
    metadata={"type": "conversation", "conversation_id": "conv_127"}
)
print(f"✅ Step 2: Stored memory with ID: {memory_id}")

# 3. Retrieve context
results = methods.retrieve_context(
    "Python programming",
    memory_store=memory,
    knowledge_graph=kg,
    vector_store=vs,
    method="hybrid",
    max_results=5
)
print(f"✅ Step 3: Retrieved {len(results)} context items")

# 4. Link entities
linked = methods.link_entities(
    entities,
    knowledge_graph=kg,
    method="knowledge_graph"
)
print(f"✅ Step 4: Linked {len(linked)} entities")

print(f"\n🎉 Complete workflow executed successfully!")


## 📚 Summary

In this notebook, we've covered:

### ✅ What We Learned

1. **Context Graph Construction**
   - Building graphs from entities, relationships, and conversations
   - Manual graph construction with nodes and edges
   - Graph traversal and querying

2. **Agent Memory Management**
   - Storing memories with metadata and entities
   - Retrieving memories with vector similarity search
   - Conversation history management
   - Memory cleanup and statistics

3. **Context Retrieval**
   - Hybrid retrieval combining multiple sources
   - Graph expansion for related entities
   - Multi-hop context discovery

4. **Entity Linking**
   - URI assignment for entities
   - Similarity-based and graph-based linking
   - Entity web construction
   - Cross-document entity linking

5. **Methods Submodule**
   - Convenient functions for all operations
   - Multiple algorithm support
   - Method discovery and retrieval

6. **Registry System**
   - Registering custom methods
   - Method management and discovery

7. **Configuration**
   - Global and method-specific settings
   - Environment variable support

### 🎯 Key Takeaways

- The context module provides a complete solution for building context-aware agents
- Multiple approaches (classes vs. methods) for different use cases
- Extensible architecture with custom method registration
- Hybrid retrieval provides the best context quality
- Entity linking enables cross-document knowledge integration

### 📖 Next Steps

- Explore the [API Reference](https://semantica.readthedocs.io/reference/context/) for detailed documentation
- Check out advanced use cases in the cookbook
- Build your own custom context methods
- Integrate with other Semantica modules for complete workflows

---

**Happy Context Engineering! 🚀**
